<a href="https://colab.research.google.com/github/pujangrg1/pujan_INFO5731_Spring2020/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

In [24]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [25]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [44]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome('chromedriver',options=options)
url ='https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
driver.get(url)
reviewsrating = []
reviewsdescription = []
reviewstitle = []
reviewsdate = []
for x in range(5):
  driver.find_element_by_class_name("ipl-load-more__button").click()
  time.sleep(6)
  revdate = driver.find_elements(By.CLASS_NAME, "review-date")
  revrating = driver.find_elements(By.CLASS_NAME, "rating-other-user-rating")
  revtitle = driver.find_elements(By.CLASS_NAME, "title")
  revdescription = driver.find_elements(By.CLASS_NAME, "text")
for a,b,c,d in zip(revdate,revrating,revtitle,revdescription):
      reviewsdate.append(a.text)
      reviewsrating.append(b.text)
      reviewstitle.append((c.text).replace('\n',''))
      reviewsdescription.append(d.text)
data = pd.DataFrame({'Date': reviewsdate,
'Rating': reviewsrating,
'Title': reviewstitle,
'Description': reviewsdescription})
data.head(100)

,Date,Rating,Title,Description
0,3 October 2019,10/10,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul..."
1,7 October 2019,10/10,Only certain people can relate,This is a movie that only those who have felt ...
2,1 September 2019,10/10,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o..."
3,1 September 2019,10/10,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...
4,4 October 2019,10/10,The Hype is real,Most of the time movies are anticipated like t...
...,...,...,...,...
95,6 December 2020,1/10,What a joke,"Heath great in dark Knight, even liked Jared L..."
96,14 January 2020,1/10,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...
97,19 October 2019,10/10,Derivative and Uninspired,There is doing an homage and then there is bor...
98,19 October 2019,1/10,Better if didn't saw,Disturbing. Just got out of the movie. My humo...


# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [43]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

**Data Preprocessing**

In [45]:
#Lowercase
data['LowerCase'] = data['Description'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data.head(100)

,Date,Rating,Title,Description,LowerCase
0,3 October 2019,10/10,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul..."
1,7 October 2019,10/10,Only certain people can relate,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...
2,1 September 2019,10/10,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o..."
3,1 September 2019,10/10,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...
4,4 October 2019,10/10,The Hype is real,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...
...,...,...,...,...,...
95,6 December 2020,1/10,What a joke,"Heath great in dark Knight, even liked Jared L...","heath great in dark knight, even liked jared l..."
96,14 January 2020,1/10,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...,i was very excited about this movie when it ca...
97,19 October 2019,10/10,Derivative and Uninspired,There is doing an homage and then there is bor...,there is doing an homage and then there is bor...
98,19 October 2019,1/10,Better if didn't saw,Disturbing. Just got out of the movie. My humo...,disturbing. just got out of the movie. my humo...


In [46]:
#Remove Punctuation
import string
data['Removed Punctuation'] = data['LowerCase'].apply(lambda x: "".join(x.lower() for x in x if x not in string.punctuation))
data.head(100)

,Date,Rating,Title,Description,LowerCase,Removed Punctuation
0,3 October 2019,10/10,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...",every once in a while a movie comes that truly...
1,7 October 2019,10/10,Only certain people can relate,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...
2,1 September 2019,10/10,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o...",truly a masterpiece the best hollywood film of...
3,1 September 2019,10/10,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...
4,4 October 2019,10/10,The Hype is real,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...
...,...,...,...,...,...,...
95,6 December 2020,1/10,What a joke,"Heath great in dark Knight, even liked Jared L...","heath great in dark knight, even liked jared l...",heath great in dark knight even liked jared le...
96,14 January 2020,1/10,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...,i was very excited about this movie when it ca...,i was very excited about this movie when it ca...
97,19 October 2019,10/10,Derivative and Uninspired,There is doing an homage and then there is bor...,there is doing an homage and then there is bor...,there is doing an homage and then there is bor...
98,19 October 2019,1/10,Better if didn't saw,Disturbing. Just got out of the movie. My humo...,disturbing. just got out of the movie. my humo...,disturbing just got out of the movie my humor ...


In [47]:
#Remove Stop Words
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['Removed StopWords'] = data['Removed Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data.head(100)

,Date,Rating,Title,Description,LowerCase,Removed Punctuation,Removed StopWords
0,3 October 2019,10/10,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...",every once in a while a movie comes that truly...,every movie comes truly makes impact joaquins ...
1,7 October 2019,10/10,Only certain people can relate,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...
2,1 September 2019,10/10,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o...",truly a masterpiece the best hollywood film of...,truly masterpiece best hollywood film 2019 one...
3,1 September 2019,10/10,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...
4,4 October 2019,10/10,The Hype is real,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,time movies anticipated like end falling short...
...,...,...,...,...,...,...,...
95,6 December 2020,1/10,What a joke,"Heath great in dark Knight, even liked Jared L...","heath great in dark knight, even liked jared l...",heath great in dark knight even liked jared le...,heath great dark knight even liked jared leto ...
96,14 January 2020,1/10,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...,i was very excited about this movie when it ca...,i was very excited about this movie when it ca...,excited movie came theaters raving reviews eve...
97,19 October 2019,10/10,Derivative and Uninspired,There is doing an homage and then there is bor...,there is doing an homage and then there is bor...,there is doing an homage and then there is bor...,homage borrowing wholesale material film littl...
98,19 October 2019,1/10,Better if didn't saw,Disturbing. Just got out of the movie. My humo...,disturbing. just got out of the movie. my humo...,disturbing just got out of the movie my humor ...,disturbing got movie humor much better seeing ...


In [48]:
#Tokenization
import nltk
nltk.download('punkt')
from textblob import TextBlob
data['Tokenization'] = data['Removed StopWords'].apply(lambda x: TextBlob(x).words)
data.head(100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Date,Rating,Title,Description,LowerCase,Removed Punctuation,Removed StopWords,Tokenization
0,3 October 2019,10/10,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...",every once in a while a movie comes that truly...,every movie comes truly makes impact joaquins ...,"[every, movie, comes, truly, makes, impact, jo..."
1,7 October 2019,10/10,Only certain people can relate,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...,"[movie, felt, alone, isolated, truly, relate, ..."
2,1 September 2019,10/10,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o...",truly a masterpiece the best hollywood film of...,truly masterpiece best hollywood film 2019 one...,"[truly, masterpiece, best, hollywood, film, 20..."
3,1 September 2019,10/10,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...,"[joaquin, phoenix, gives, tour, de, force, per..."
4,4 October 2019,10/10,The Hype is real,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,time movies anticipated like end falling short...,"[time, movies, anticipated, like, end, falling..."
...,...,...,...,...,...,...,...,...
95,6 December 2020,1/10,What a joke,"Heath great in dark Knight, even liked Jared L...","heath great in dark knight, even liked jared l...",heath great in dark knight even liked jared le...,heath great dark knight even liked jared leto ...,"[heath, great, dark, knight, even, liked, jare..."
96,14 January 2020,1/10,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...,i was very excited about this movie when it ca...,i was very excited about this movie when it ca...,excited movie came theaters raving reviews eve...,"[excited, movie, came, theaters, raving, revie..."
97,19 October 2019,10/10,Derivative and Uninspired,There is doing an homage and then there is bor...,there is doing an homage and then there is bor...,there is doing an homage and then there is bor...,homage borrowing wholesale material film littl...,"[homage, borrowing, wholesale, material, film,..."
98,19 October 2019,1/10,Better if didn't saw,Disturbing. Just got out of the movie. My humo...,disturbing. just got out of the movie. my humo...,disturbing just got out of the movie my humor ...,disturbing got movie humor much better seeing ...,"[disturbing, got, movie, humor, much, better, ..."


In [49]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
data['Stemming'] = data['Tokenization'].apply(lambda x: " ".join([st.stem(word) for word in x]))
data.head(100)

,Date,Rating,Title,Description,LowerCase,Removed Punctuation,Removed StopWords,Tokenization,Stemming
0,3 October 2019,10/10,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...",every once in a while a movie comes that truly...,every movie comes truly makes impact joaquins ...,"[every, movie, comes, truly, makes, impact, jo...",everi movi come truli make impact joaquin perf...
1,7 October 2019,10/10,Only certain people can relate,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...,"[movie, felt, alone, isolated, truly, relate, ...",movi felt alon isol truli relat understand mot...
2,1 September 2019,10/10,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o...",truly a masterpiece the best hollywood film of...,truly masterpiece best hollywood film 2019 one...,"[truly, masterpiece, best, hollywood, film, 20...",truli masterpiec best hollywood film 2019 one ...
3,1 September 2019,10/10,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...,"[joaquin, phoenix, gives, tour, de, force, per...",joaquin phoenix give tour de forc perform fear...
4,4 October 2019,10/10,The Hype is real,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,time movies anticipated like end falling short...,"[time, movies, anticipated, like, end, falling...",time movi anticip like end fall short way shor...
...,...,...,...,...,...,...,...,...,...
95,6 December 2020,1/10,What a joke,"Heath great in dark Knight, even liked Jared L...","heath great in dark knight, even liked jared l...",heath great in dark knight even liked jared le...,heath great dark knight even liked jared leto ...,"[heath, great, dark, knight, even, liked, jare...",heath great dark knight even like jare leto su...
96,14 January 2020,1/10,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...,i was very excited about this movie when it ca...,i was very excited about this movie when it ca...,excited movie came theaters raving reviews eve...,"[excited, movie, came, theaters, raving, revie...",excit movi came theater rave review even excit...
97,19 October 2019,10/10,Derivative and Uninspired,There is doing an homage and then there is bor...,there is doing an homage and then there is bor...,there is doing an homage and then there is bor...,homage borrowing wholesale material film littl...,"[homage, borrowing, wholesale, material, film,...",homag borrow wholesal materi film littl remak ...
98,19 October 2019,1/10,Better if didn't saw,Disturbing. Just got out of the movie. My humo...,disturbing. just got out of the movie. my humo...,disturbing just got out of the movie my humor ...,disturbing got movie humor much better seeing ...,"[disturbing, got, movie, humor, much, better, ...",disturb got movi humor much better see first f...


In [50]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
#Lemmatization
from textblob import Word
data['Lemmatization'] = data['Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data.head(100)

,Date,Rating,Title,Description,LowerCase,Removed Punctuation,Removed StopWords,Tokenization,Stemming,Lemmatization
0,3 October 2019,10/10,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...",every once in a while a movie comes that truly...,every movie comes truly makes impact joaquins ...,"[every, movie, comes, truly, makes, impact, jo...",everi movi come truli make impact joaquin perf...,everi movi come truli make impact joaquin perf...
1,7 October 2019,10/10,Only certain people can relate,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...,"[movie, felt, alone, isolated, truly, relate, ...",movi felt alon isol truli relat understand mot...,movi felt alon isol truli relat understand mot...
2,1 September 2019,10/10,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o...",truly a masterpiece the best hollywood film of...,truly masterpiece best hollywood film 2019 one...,"[truly, masterpiece, best, hollywood, film, 20...",truli masterpiec best hollywood film 2019 one ...,truli masterpiec best hollywood film 2019 one ...
3,1 September 2019,10/10,MASTERPIECE 😍,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...,"[joaquin, phoenix, gives, tour, de, force, per...",joaquin phoenix give tour de forc perform fear...,joaquin phoenix give tour de forc perform fear...
4,4 October 2019,10/10,The Hype is real,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,time movies anticipated like end falling short...,"[time, movies, anticipated, like, end, falling...",time movi anticip like end fall short way shor...,time movi anticip like end fall short way shor...
...,...,...,...,...,...,...,...,...,...,...
95,6 December 2020,1/10,What a joke,"Heath great in dark Knight, even liked Jared L...","heath great in dark knight, even liked jared l...",heath great in dark knight even liked jared le...,heath great dark knight even liked jared leto ...,"[heath, great, dark, knight, even, liked, jare...",heath great dark knight even like jare leto su...,heath great dark knight even like jare leto su...
96,14 January 2020,1/10,Glad I didn't waste my money on this movie,I was very excited about this movie when it ca...,i was very excited about this movie when it ca...,i was very excited about this movie when it ca...,excited movie came theaters raving reviews eve...,"[excited, movie, came, theaters, raving, revie...",excit movi came theater rave review even excit...,excit movi came theater rave review even excit...
97,19 October 2019,10/10,Derivative and Uninspired,There is doing an homage and then there is bor...,there is doing an homage and then there is bor...,there is doing an homage and then there is bor...,homage borrowing wholesale material film littl...,"[homage, borrowing, wholesale, material, film,...",homag borrow wholesal materi film littl remak ...,homag borrow wholesal materi film littl remak ...
98,19 October 2019,1/10,Better if didn't saw,Disturbing. Just got out of the movie. My humo...,disturbing. just got out of the movie. my humo...,disturbing just got out of the movie my humor ...,disturbing got movie humor much better seeing ...,"[disturbing, got, movie, humor, much, better, ...",disturb got movi humor much better see first f...,disturb got movi humor much better see first f...


In [52]:
import csv
data.to_csv("MovieReviews.csv", index= False)

In [53]:
#Q.1.1. Count the frequency of all the N-grams (N=3).
text_frequency = (data['Lemmatization']).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
text_frequency.columns = ['Words','Frequency']
text_frequency.head(100)

,Words,Frequency
0,watch,51.0
1,emot,9.0
2,time,46.0
3,movi,289.0
4,haunt,2.0
...,...,...
95,reflect,1.0
96,someth,13.0
97,find,7.0
98,talent,5.0


In [54]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

wordSen = []
for sentence in data['Lemmatization']:
  wordSen.append(word_tokenize(sentence))

aftercleaning = [x for x in wordSen if x != []]
iterations = list(itertools.chain.from_iterable(aftercleaning))

In [55]:
t_grams = nltk.trigrams(iterations)
f_dist = nltk.FreqDist(t_grams)
f_dist

FreqDist({('everi', 'movi', 'come'): 1,
          ('movi', 'come', 'truli'): 1,
          ('come', 'truli', 'make'): 1,
          ('truli', 'make', 'impact'): 1,
          ('make', 'impact', 'joaquin'): 1,
          ('impact', 'joaquin', 'perform'): 1,
          ('joaquin', 'perform', 'scenographi'): 1,
          ('perform', 'scenographi', 'brillianc'): 1,
          ('scenographi', 'brillianc', 'grotesqu'): 1,
          ('brillianc', 'grotesqu', 'haunt'): 1,
          ('grotesqu', 'haunt', 'cringi'): 1,
          ('haunt', 'cringi', 'hard'): 1,
          ('cringi', 'hard', 'watch'): 1,
          ('hard', 'watch', 'time'): 1,
          ('watch', 'time', 'mesmer'): 1,
          ('time', 'mesmer', 'wont'): 1,
          ('mesmer', 'wont', 'blink'): 1,
          ('wont', 'blink', 'eye'): 1,
          ('blink', 'eye', 'watch'): 1,
          ('eye', 'watch', 'tragic'): 1,
          ('watch', 'tragic', 'serious'): 1,
          ('tragic', 'serious', 'funni'): 1,
          ('serious', 'funni', '

**Q.1.2.Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.**


In [56]:
from collections import Counter
bigrams = nltk.bigrams(iterations)
freq_dist = nltk.FreqDist(bigrams)
bigrams_dict = dict(freq_dist)
for word in bigrams_dict:
  print( str(word) + ':' + str(bigrams_dict[word] / iterations.count(word[0])))

Streaming output truncated to the last 5000 lines.
('seem', 'gotham'):0.08333333333333333
('gotham', 'citizen'):0.125
('citizen', 'entir'):0.5
('entir', 'univers'):0.16666666666666666
('univers', 'matter'):0.1111111111111111
('matter', 'exist'):0.5
('exist', 'put'):0.25
('put', 'arthur'):0.07142857142857142
('arthur', 'consid'):0.05
('consid', 'film'):0.1111111111111111
('film', 'isnt'):0.010638297872340425
('isnt', 'told'):0.1
('told', 'arthur'):0.2
('arthur', 'perspect'):0.05
('perspect', 'blatantli'):1.0
('blatantli', 'stupid'):1.0
('stupid', 'depict'):0.2
('world', 'reason'):0.09090909090909091
('reason', 'peopl'):0.2
('like', 'much'):0.014705882352941176
('much', 'sad'):0.03225806451612903
('sad', 'corpor'):0.2
('corpor', 'slave'):1.0
('slave', 'especi'):1.0
('especi', 'youth'):0.3333333333333333
('youth', 'current'):1.0
('current', 'indoctrin'):1.0
('indoctrin', 'subconsci'):1.0
('subconsci', 'trick'):1.0
('trick', 'believ'):1.0
('believ', 'part'):0.058823529411764705
('part', 'b

**Q.1.3.Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).**

In [57]:
dataSet = ''
index = []
i = 1
for line in data['Lemmatization']:
  dataSet = dataSet + line
  stringVal = 'Review :' + str(i)
  index.append(stringVal)
  i += 1

In [61]:
from textblob import TextBlob
nounPhrases = []
freq = []
for line in data['Lemmatization']:
  b = TextBlob(line)
  for nouns in b.noun_phrases:
    nounPhrases.append(nouns)
for word in nounPhrases:
  nounPhrases_freq = []
  for line in data['Lemmatization']:
    nounPhrases_freq.append(line.count(word) / dataSet.count(word))
  freq.append(nounPhrases_freq)
nounPhrases_df = pd.DataFrame(freq).T
nounPhrases_df.columns = list(nounPhrases)
nounPhrases_df.index = index
nounPhrases_df

,everi movi,impact joaquin,scenographi brillianc grotesqu haunt cringi,time mesmer wont blink eye,serious funni moment emot rollercoast sometim multipl emot poppingup time,typic actionriddl,superhero movi,proper psycholog thrillerdrama singl,alon isol truli relat,sorri charact lot peopl,encourag violenc truli movi encourag everi,u becom,everyon respect,truli masterpiec,hollywood film,film decad truli,comic book chillingli realist,real ife remark,direct cinematographi music act peopl surpris,violent necess messag societi reflect underappreciatedunrecognizedbulli peopl,someth way show class differ,corrupt rich talent rule,differ believ,multipl joker,bitter way film show,peopl uncomfort peopl consid wake,perfect film,joaquin phoenix,fearless stun emot depth physic imposs,referenc heath ledger oscarwin,dark knight,wide consid definit liveact portray joker,fact everyon,stun phoenix,imposs portray,potenti exce,dark knight clown princ crime,time movi anticip,end fall,short way,...,spectacular hit 70´ 80´ vote,titl brazil coringa joker,basic bad remak taxi driver bit king comedi thrown replac underag prostitut part oh,littl batman stuff link,popular superhero movi problem unlik taxi driver,main charact unapp unlik,hard sympathis,wrong bronson style,american psycho,basic innoc peopl fun glamoris violenc inoc peopl,mental disturb,great messag,time wast phoenix danc mug camera,long bore,hype movi,violent movi wasnt nearli good,doubt act good,dark depress movi guy,bipolar disord,includ mother kind neg messag pas societi,sick dont,old miser man hollywood,time time name,blade runner,star war toy stori,aveng infin war endgam etcetera etcetera,disappoint disappoint,joker hope,appar way,good review hype troll medium peopl opinion wallet easi bore shallow storylin,plot whatsoev,scene quot empti nada,opposit plot movi dont,sen socal plot,bad bad compromis check box law hollywood peopl fall wonder im,joker joke,hollywood joke,review system joke,realiz cinema peopl,review troll
Review :1,1.0,1.0,1.0,1.0,1.0,1.0,0.111111,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review :2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review :3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review :4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025641,1.0,1.0,0.166667,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Review :5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

**(1) To build the documents-terms weights (tf*idf) matrix bold text.**

In [64]:
# Write your code here
import math
def tfidf_calculator(row,word):
  a = len(row.split(" "))
  tf_val = row.count(word)/a
  idf_val = 0
  if(tf_val!=0):
    idf_val = math.log(a)/row.count(word)
  else:
    return 0;
  return tf_val*idf_val

rows = data["Removed StopWords"].values.tolist()
tokens = set([i for j in rows for i in j.split(" ")])
tfidf_df = pd.DataFrame(tokens,columns=["Token"])
count = 0
for x in rows:
  tfidf_df[str(count)] = tfidf_df["Token"].apply(lambda i : tfidf_calculator(x,i))
  count+=1

tfidf_df




,Token,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146
0,,0.081918,0.093937,0.055855,0.08065,0.083231,0.058163,0.110774,0.035297,0.070931,0.0,0.0,0.061364,0.188504,0.244136,0.074911,0.040702,0.244136,0.071881,0.0,0.058773,0.074911,0.064221,0.128755,0.020991,0.0,0.039377,0.069111,0.0,0.0,0.072861,0.054775,0.105955,0.093937,0.298627,0.08065,0.116114,0.0,0.093937,0.0,...,0.0,0.0,0.188504,0.0,0.149787,0.083231,0.088992,0.022682,0.188504,0.0,0.321888,0.0,0.128755,0.04957,0.0,0.084592,0.067391,0.21799,0.125311,0.0,0.0,0.092222,0.064221,0.15497,0.044658,0.0,0.093937,0.0,0.16666,0.0,0.180537,0.059398,0.188504,0.103716,0.050434,0.057567,0.101581,0.0,0.0,0.04126
1,pretty,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000
2,shows,0.000000,0.000000,0.055855,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.072861,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.093937,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.050434,0.000000,0.000000,0.0,0.0,0.00000
3,card,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.088992,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000
4,happiness,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.022682,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2581,overhyped,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071881,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.039377,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.298627,0.00000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.00

**(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using cosine similarity.**

In [73]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
query = "An Outstanding movie with a haunting performance and best character development."
a = word_tokenize(query)
s = stopwords.words('english') 
setX = {w for w in a if not w in s}

In [75]:
def vector(setX, setY):
  v = setX.union(setY)
  firstList =[]
  secondList =[]
  for i in v: 
      if i in setX: firstList.append(1)
      else: firstList.append(0) 
      if i in setY: secondList.append(1) 
      else: secondList.append(0)
  return v, firstList, secondList

def cos(v, firstList, secondList):
  a = 0
  for i in range(len(v)): 
    a += firstList[i] * secondList[i] 
  cos = a/float((sum(firstList)*sum(secondList))**0.5) 
  return cos

In [77]:
newList = []
for row in data['Removed StopWords']:
  ListA = word_tokenize(row)
  setY = {i for i in ListA if not i in x}
  v, firstList, secondList = vector(setX, setY)
  try:
    new = cos(v, firstList, secondList)
  except ZeroDivisionError:
    new = 'None'
  newList.append(new)
cosine_sim_df = pd.DataFrame(list(zip(data['Description'],newList)), columns=['Reviews','Cosine Similarity'])
cosine_sim_df

,Reviews,Cosine Similarity
0,"Every once in a while a movie comes, that trul...",0.257172
1,This is a movie that only those who have felt ...,0.065372
2,"Truly a masterpiece, The Best Hollywood film o...",0.0453609
3,Joaquin Phoenix gives a tour de force performa...,0.0533761
4,Most of the time movies are anticipated like t...,0
...,...,...
142,Basically a bad remake of taxi driver with bit...,0.0419961
143,I just didn't get the hype about this movie. A...,0
144,,None
145,,None


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [79]:
import csv
import pandas as pd
moviewReviews = pd.read_csv('MovieReviews.csv', encoding = "ISO-8859-1")
moviewReviews

#Link to file in GitHub
#https://github.com/pujangrg1/pujan_INFO5731_Spring2020/blob/main/MovieReviews.csv

,Doc_Id,Review,Cleaned Data,Sentimental Analysis
0,Review 1,"Every once in a while a movie comes, that trul...",everi movi come truli make impact joaquin perf...,Positive
1,Review 2,This is a movie that only those who have felt ...,movi felt alon isol truli relat understand mot...,Positive
2,Review 3,"Truly a masterpiece, The Best Hollywood film o...",truli masterpiec best hollywood film 2019 one ...,Positive
3,Review 4,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix give tour de forc perform fear...,Neutral
4,Review 5,Most of the time movies are anticipated like t...,time move anticip like end fall short way shor...,Negative
...,...,...,...,...
103,Review 104,"Truly a masterpiece, The Best Hollywood film o...",truli masterpiec best hollywood film 2019 one ...,Positive
104,Review 105,"The acting, cinematography, sound design and t...",act cinematographi sound design script phenome...,Positive
105,Review 106,If you plan on watching this I would suggest s...,plan watch would suggest ship last 20 30min ot...,Negative
106,Review 107,"If it wasn't for the title and make-up, you wo...",want titl make wouldn know base comicbook char...,Positive
